In [227]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [2]:
from pygam import LinearGAM

In [119]:
import statsmodels.api as sm

In [3]:
from scipy.interpolate import interp1d

In [4]:
train = pd.read_csv("covid19-global-forecasting-week-2/train.csv")

In [6]:
train.columns

Index(['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities'],
      dtype='object')

In [5]:
test = pd.read_csv("covid19-global-forecasting-week-2/test.csv")

In [27]:
train['Country_State'] = train['Country_Region'].str.cat(train.Province_State.fillna(''),sep=" ")

In [30]:
test['Country_State'] = test['Country_Region'].str.cat(test.Province_State.fillna(''),sep=" ")

In [114]:
train['Country_State'].unique()

array(['Afghanistan ', 'Albania ', 'Algeria ', 'Andorra ', 'Angola ',
       'Antigua and Barbuda ', 'Argentina ', 'Armenia ',
       'Australia Australian Capital Territory',
       'Australia New South Wales', 'Australia Northern Territory',
       'Australia Queensland', 'Australia South Australia',
       'Australia Tasmania', 'Australia Victoria',
       'Australia Western Australia', 'Austria ', 'Azerbaijan ',
       'Bahamas ', 'Bahrain ', 'Bangladesh ', 'Barbados ', 'Belarus ',
       'Belgium ', 'Belize ', 'Benin ', 'Bhutan ', 'Bolivia ',
       'Bosnia and Herzegovina ', 'Brazil ', 'Brunei ', 'Bulgaria ',
       'Burkina Faso ', 'Cabo Verde ', 'Cambodia ', 'Cameroon ',
       'Canada Alberta', 'Canada British Columbia', 'Canada Manitoba',
       'Canada New Brunswick', 'Canada Newfoundland and Labrador',
       'Canada Nova Scotia', 'Canada Ontario',
       'Canada Prince Edward Island', 'Canada Quebec',
       'Canada Saskatchewan', 'Central African Republic ', 'Chad ',
    

## aggregate level model

In [234]:
train_use = train[train['Date'] > '2020-01-21']

In [241]:
train_use['Date_diff'] = (pd.to_datetime(train_use['Date']) - pd.to_datetime(train_use['Date'].min())).dt.days
train_use['Date_diff'] = train_use['Date_diff']+1

In [237]:
train_use = train_use[train_use['Date_diff'] > 0]

In [200]:
train_agg = train_use.groupby('Date_diff')['ConfirmedCases','Fatalities'].sum().reset_index()

In [207]:
# model for confirmed cases
cc_model_agg = LinearGAM(n_splines=10).fit(train_agg['Date_diff'], train_agg['ConfirmedCases'])
train_agg['pred_cc'] = cc_model_agg.predict(train_agg['Date_diff'])

r_square_cc = round(r2_score(train_agg['ConfirmedCases'], train_agg['pred_cc']),3)*100

# model for fatalities
fa_model_agg = sm.OLS(train_agg['Fatalities'], train_agg['ConfirmedCases']).fit()
train_agg['pred_fa'] = fa_model_agg.predict(train_agg['ConfirmedCases'])

r_square_fa = round(r2_score(train_agg['Fatalities'], train_agg['pred_fa']),3)*100

## country level model

In [253]:
def data_manipulation(Country_State):
    trn = train[(train['Country_State'] == Country_State)]
    tst = test[(test['Country_State'] == Country_State)]
    
    #date for first recorded case
    st_dt = trn['Date'][trn['ConfirmedCases'] > 0].min()
    
    trn['Date_diff'] = (pd.to_datetime(trn['Date']) - pd.to_datetime(st_dt)).dt.days
    trn['Date_diff'] = trn['Date_diff'] + 1
    trn = trn[trn['Date_diff'] > 0]
    
    tst['Date_diff'] = (pd.to_datetime(tst['Date']) - pd.to_datetime(st_dt)).dt.days
    tst['Date_diff'] = tst['Date_diff'] + 1
    
    return(trn, tst)

In [254]:
train_model_accuracy = pd.DataFrame(columns = ['Country_State','r_square_cc','r_square_fa'])
test_update = pd.DataFrame(columns = ['ForecastId', 'Province_State', 'Country_Region', 'Date', 'Country_State', 'pred_cc','pred_fa'])

for Country_State in test['Country_State'].unique():
    print(Country_State)
    
    trn, tst = data_manipulation(Country_State)
    
    if len(trn) > 10:
#         model for confirmed cases
        cc_model = LinearGAM(n_splines=10).fit(trn['Date_diff'], trn['ConfirmedCases'])
        trn['pred_cc'] = cc_model.predict(trn['Date_diff'])
        tst['pred_cc'] = cc_model.predict(tst['Date_diff'])
#         print(gam.coef_)

#         model for fatalities
        fa_model = sm.OLS(trn['Fatalities'], trn['ConfirmedCases']).fit()
        trn['pred_fa'] = fa_model.predict(trn['ConfirmedCases'])
        tst['pred_fa'] = fa_model.predict(tst['pred_cc'])


    else:
        print('less than 10 rows.. fall back to agg model')
#         use aggregated model for confirmed cases
        trn['pred_cc'] = cc_model_agg.predict(trn['Date_diff'])
        tst['pred_cc'] = cc_model_agg.predict(tst['Date_diff'])
#         print(gam.coef_)

#         use aggregated model for fatalities
        trn['pred_fa'] = fa_model_agg.predict(trn['ConfirmedCases'])
        tst['pred_fa'] = fa_model_agg.predict(tst['pred_cc'])
        
    r_square_cc = round(r2_score(trn['ConfirmedCases'], trn['pred_cc']),3)*100
    r_square_fa = round(r2_score(trn['Fatalities'], trn['pred_fa']),3)*100

#         plot confirmed cases
    fig = plt.figure()
    plt.ioff()
    fig.set_size_inches(18.5, 10.5)
    plt.plot(trn['Date_diff'], trn['ConfirmedCases'], marker='o')
    plt.plot(trn['Date_diff'], trn['pred_cc'])
    plt.plot(tst['Date_diff'], tst['pred_cc'], linestyle='--')
    fig.suptitle(Country_State, fontsize=20)
    fig.text(0.15, 0.93,'r square for confirmed cases: '+str(r_square_cc)+'%', fontsize=15)

#         plot fatalities
    plt.plot(trn['Date_diff'], trn['Fatalities'], marker='s')
    plt.plot(trn['Date_diff'], trn['pred_fa'])
    plt.plot(tst['Date_diff'], tst['pred_fa'], linestyle='--')
    fig.suptitle(Country_State, fontsize=20)
    fig.text(0.15, 0.89,'r square for fatalities: '+str(r_square_fa)+'%', fontsize=15)
    fig.savefig('plots/gams/'+Country_State+'.jpg')
    plt.close(fig)

    train_model_accuracy = train_model_accuracy.append({'Country_State': Country_State, 'r_square_cc': r_square_cc, 'r_square_fa': r_square_fa}, ignore_index=True)
    test_update = test_update.append(tst[['ForecastId', 'Province_State', 'Country_Region', 'Date', 'Country_State','pred_cc','pred_fa']], ignore_index=True)


Afghanistan 


/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Albania 


/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Algeria 


...

/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Zambia 


/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Zimbabwe 
less than 10 rows.. fall back to agg model


/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [260]:
test_update['pred_cc'][test_update['pred_cc'] < 0] = 0.0
test_update['pred_fa'][test_update['pred_fa'] < 0] = 0.0

/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/z001t72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [261]:
train_model_accuracy.to_csv('train_model_accuracy.csv',index=False)
test_update.to_csv('test_update.csv',index=False)

1. treat for missing countries
2. treat for countries without fatalities
3. treat for poor accuracy
4. treat for negative values
5. treat for very high values